In [1]:
#!pip install transformers einops accelerate bitsandbytes

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import pipeline

import torch

import base64



/Users/m0k08nh/projects/preparing/test/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
checkpoint = "MBZUAI/LaMini-T5-738M"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
base_model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [5]:
from langchain_huggingface import HuggingFacePipeline

def slm_pipeline():
    pipe = pipeline(
        "text2text-generation",
        model = base_model,
        tokenizer = tokenizer,
        max_length = 256,
        do_sample = True,
        temperature = 0.3,
        top_p = 0.95
    )
    local_slm = HuggingFacePipeline(pipeline = pipe)
    return local_slm

In [7]:
input_prompt = "Write an article about Blockchain and its benefits"

In [6]:
model = slm_pipeline()

Device set to use mps:0


In [8]:
gen_text = model(input_prompt)
#gen_text = model.invoke(input_prompt)

/var/folders/8p/d5kbr2ws10x8yxm5dk18ts_40000gp/T/ipykernel_5987/3568923136.py:1: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  gen_text = model(input_prompt)


In [9]:
gen_text

'Blockchain is a decentralized digital ledger that records transactions in a secure and transparent manner. It is a decentralized system that allows for secure and transparent transactions without the need for intermediaries like banks or governments. Blockchain has numerous benefits, including: 1. Security: Blockchain technology provides a high level of security as it is designed to protect sensitive information from unauthorized access. This makes it ideal for applications such as financial transactions, supply chain management, and voting systems. 2. Transparency: Blockchain is transparent, which means that it is not influenced by any one person or entity. This means that it is impossible to'

In [11]:
!pip show langchain

Name: langchain
Version: 0.3.24
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /Users/m0k08nh/projects/preparing/test/venv/lib/python3.12/site-packages
Requires: langchain-core, langchain-text-splitters, langsmith, pydantic, PyYAML, requests, SQLAlchemy
Required-by: langchain-community, langchain-neo4j, langchain-tavily


In [ ]:
!pip install transformers einops accelerate bitsandbytes
from transformers import pipeline

import torch
import base64

checkpoint = "MBZUAI/LaMini-T5-738M"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
base_model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

!pip install langchain langchain-community langchain-huggingface

from langchain_huggingface import HuggingFacePipeline

def slm_pipeline():
    pipe = pipeline(
        "text2text-generation",
        model = base_model,
        tokenizer = tokenizer,
        max_length = 256,
        do_sample = True,
        temperature = 0.3,
        top_p = 0.95
    )
    local_slm = HuggingFacePipeline(pipeline = pipe)
    return local_slm
    


input_prompt = "Write an article about Blockchain and its benefits"

model = slm_pipeline()
gen_text = model.invoke(input_prompt)
gen_text


import json
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

# Hub Model configuration. https://huggingface.co/models
hub = {
	'HF_MODEL_ID':'MBZUAI/LaMini-T5-738M',
	'HF_TASK' : 'text2text-generation',
    'device_map' : 'auto',
    'torch_dtype' : 'torch.float32'
}



# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	image_uri=get_huggingface_llm_image_uri("huggingface",version="3.2.3"),
	env=hub,
	role=role, 
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1,
	instance_type="ml.g5.xlarge",
	container_startup_health_check_timeout=300,
  )
  
# send request
predictor.predict({
	"inputs": "Write an article about Cyber Security",
})

ENDOINT = "your endpoint"

import boto3

sagemaker_runtime = boto3.client("sagemaker-runtime", region_name='us-east-1')

endpoint_name = ENDPOINT

# API Payload
prompt = "Write an article on Deep learning"

payload = {
    'inputs' : prompt ,
    'parameters' : {
        'max_new_tokens' : 256,
        'do_sample' : True,
        'temperature' : 0.3,
        'top_p' : 0.7,
        'top_k' : 50,
        'repetion_penalty' : 1.03
    }
}

response = sagemaker_runtime.invoke_endpoint(
    EndpointName = endpoint_name,
    ContentType = "application/json",
    Body = json.dumps(payload)
)

predictions = json.loads(response['Body'].read().decode('utf-8'))
final_result =predictions[0]['generated_text']


##############LAMBDA###############

import json
import boto3

ENDPOINT = "80"

sagemaker_runtime = boto3.client("sagemaker-runtime", region_name='us-east-1')

def lambda_handler(event, context):
    query_params = event['queryStringParameters']
    query = query_params['query']
    payload = {
        "inputs": query,
        "parameters": {
            "max_new_tokens": 256,
            "top_p": 0.9,
            "temperature": 0.6,
            "top_k": 50,
            "repetion_penalty" : 1.03,
            "do_sample" : True
        }
    }
    response = sagemaker_runtime.invoke_endpoint(EndpointName=ENDPOINT,
                                                  ContentType="application/json",
                                                  Body=json.dumps(payload))
    predictions = json.loads(response['Body'].read().decode('utf-8'))
    final_result = predictions[0]['generated_text']
    return {
        'statusCode': 200,
        'body': json.dumps(final_result)
    }

















